In [1]:
import pandas as pd
import numpy as np

In [1]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
# urllib = urllib3.PoolManager()
# url = 'http://www.manythings.org/anki/fra-eng.zip'
# filename = 'fra-eng.zip'
# path = os.getcwd()
# zipfilename = os.path.join(path, filename)

# with urllib.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
#     shutil.copyfileobj(r, out_file)

# with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
#     zip_ref.extractall(path)

In [4]:
lines = pd.read_csv('fra.txt', names=['src','tar','lic'],sep='\t')
del lines['lic']
lines.sample(10)

,src,tar
43445,Maybe we should pray.,Peut-être devrions-nous prier.
134042,She works as hard as anybody does.,Elle travaille aussi dur que n'importe qui.
119129,Do you know how to ride a horse?,Savez-vous monter à cheval ?
172061,"Will it stop raining soon? ""I'm afraid not.""",« Est-ce qu'il va bientôt s'arrêter de pleuvoi...
41855,I like that necklace.,J'aime ce collier.
48010,How could you do that?,Comment as-tu pu faire ça ?
109324,Please help me take this down.,Veuillez m'aider à descendre ceci.
26841,What a smart idea!,Quelle idée judicieuse !
97345,Why did she come home early?,Pourquoi est-elle rentrée tôt à la maison ?
111692,You can't get there from here.,On ne peut pas y parvenir d'ici.


In [5]:
len(lines)

192341

In [6]:
# 전처리 1. 앞뒤 토큰붙여주고 2. 알파벳별로 모아서 인덱싱 3. encoder_input 만들기

In [7]:
lines = lines.iloc[:60000,:]
lines

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,Bouge !
3,Hi.,Salut !
4,Hi.,Salut.
...,...,...
59995,Why would they do that?,Pourquoi feraient-elles ça ?
59996,Why would you ask that?,Pourquoi demanderais-tu cela ?
59997,Why would you ask that?,Pourquoi demanderiez-vous cela ?
59998,Why would you say that?,Pourquoi dirais-tu cela ?


In [8]:
lines['tar'] = lines['tar'].apply(lambda x : '\t '+ x + ' \n' )
lines.sample(10)

,src,tar
17847,He was all wrong.,\t Il avait tout faux. \n
41617,I had lunch with Tom.,\t J'ai déjeuné avec Tom. \n
20705,Tom is terrified.,\t Tom est terrifié. \n
40857,He looks like a girl.,\t Il a l'air d'une fille. \n
34197,Don't you like fish?,\t Vous n'aimez pas le poisson ? \n
23662,I like your house.,\t J'aime ta maison. \n
54311,Everyone remained calm.,\t Tout le monde est resté calme. \n
24480,Is that all clear?,\t Est-ce entièrement clair ? \n
26286,Tom never gave up.,\t Tom ne renonça jamais. \n
27002,Where's my sister?,\t Où est ma sœur ? \n


In [9]:
 # 2. 알파벳별로 모아서 인덱싱 

In [10]:
src_vocab = set() # 중복피하기 위해

for line in lines['src']:
    for char in line:
        src_vocab.add(char)
# src_vocab[:10] set순서가 없어서 인덱싱 불가능

In [11]:
tar_vocab = set()
for line in lines['tar']:
    for char in line:
        tar_vocab.add(char)

In [12]:
# 단어사이즈 확인
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1

print(src_vocab_size)
print(tar_vocab_size)

80
105


In [13]:
# 인덱싱을 위해 정렬
src_vocab_sorted = sorted(list(src_vocab))
tar_vocab_sorted = sorted(list(tar_vocab))

src_vocab_sorted[20:30]


['9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G']

In [16]:
# 인덱싱

src_to_index = dict([(word, i+1) for i,word in enumerate(src_vocab_sorted)])
tar_to_index = dict([(word, i+1) for i,word in enumerate(tar_vocab_sorted)])
print(src_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '°': 76, 'é': 77, '’': 78, '€': 79}


In [26]:
# encoder input 

encoder_input = []
for line in lines['src']:
    temp = []
    for char in line:
        temp.append(src_to_index[char])
    encoder_input.append(temp)
encoder_input[:5]

[[30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10], [31, 58, 10]]

In [25]:
# decoder input
decoder_input= []
for line in lines['tar']:
    temp = []
    for char in line:
        temp.append(tar_to_index[char])
    decoder_input.append(temp)
decoder_input[:5]

[[1, 3, 48, 53, 3, 4, 3, 2],
 [1, 3, 39, 53, 70, 55, 60, 57, 14, 3, 2],
 [1, 3, 28, 67, 73, 59, 57, 3, 4, 3, 2],
 [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2],
 [1, 3, 45, 53, 64, 73, 72, 14, 3, 2]]

In [27]:
# decoder target 첫번째 <sos>제외 시켜줘야함

decoder_target = []
for line in lines['tar']:
    temp = []
    for char in line[1:]:
        temp.append(tar_to_index[char])
    decoder_target.append(temp)
    
decoder_target[:5]

[[3, 48, 53, 3, 4, 3, 2],
 [3, 39, 53, 70, 55, 60, 57, 14, 3, 2],
 [3, 28, 67, 73, 59, 57, 3, 4, 3, 2],
 [3, 45, 53, 64, 73, 72, 3, 4, 3, 2],
 [3, 45, 53, 64, 73, 72, 14, 3, 2]]

In [33]:
# 인코딩한 input, target값들 길이 맞춰주려면 최댓값을 알아야 함

max_len_enc = max([len(x) for x in encoder_input])
max_len_dec_inp = max([len(x) for x in decoder_input])
max_len_dec_tar = max([len(x) for x in decoder_target])
print(max_len_enc)
print(max_len_dec_inp)
print(max_len_dec_tar)

23
76
75


In [34]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input_pad = pad_sequences(encoder_input,maxlen=max_len_enc, padding='post')
decoder_input_pad = pad_sequences(decoder_input,maxlen=max_len_dec_inp, padding='post')
decoder_target_pad = pad_sequences(decoder_target,maxlen=max_len_dec_tar, padding='post')